# Extract Data

In [ ]:
import pandas as pd  # type: ignore # Import the Pandas library for data manipulation and analysis

# Read the Excel file into a Pandas DataFrame
df = pd.read_excel("transactions.xlsx", engine="openpyxl")  
# "transactions.xlsx" is the file containing transaction data
# The "engine='openpyxl'" parameter ensures compatibility with modern Excel file formats (.xlsx)

# Display the first few rows of the DataFrame
df.head()  # Shows the first 5 rows of the dataset for quick inspection


# Transform Data

In [ ]:
# Remove the last unnamed column (".")
df = df.iloc[:, :-1]  
# The last column in the dataset appears to be an unwanted/empty column, so we drop it by selecting all columns except the last one.

# Clean the 'Account No' column
df["Account No"] = df["Account No"].astype(str).str.replace("'", "")  
# Convert 'Account No' column to string type to ensure uniformity
# Remove any single quotes (') present in the 'Account No' column to clean the data


In [ ]:
# Convert date columns to datetime format
df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")  
# Convert the 'DATE' column to datetime format for proper date-based analysis
# The 'errors="coerce"' argument ensures that any invalid date formats are converted to NaT (Not a Time)

df["VALUE DATE"] = pd.to_datetime(df["VALUE DATE"], errors="coerce")  
# Convert the 'VALUE DATE' column to datetime format for consistency and easier date-based operations
# 'errors="coerce"' handles any incorrect or missing date values by converting them to NaT


In [ ]:
# Fill missing values in the 'CHQ.NO.' column with "Not Available"
# This ensures that blank cheque numbers are replaced with a readable value instead of NaN.
df["CHQ.NO."] = df["CHQ.NO."].fillna("Not Available")

# Replace NaN (missing values) in the 'WITHDRAWAL AMT' column with 0
# This ensures that missing withdrawal amounts are treated as zero instead of NaN.
df["WITHDRAWAL AMT"] = df["WITHDRAWAL AMT"].fillna(0)

# Replace NaN (missing values) in the 'DEPOSIT AMT' column with 0
# This ensures that missing deposit amounts are treated as zero instead of NaN.
df["DEPOSIT AMT"] = df["DEPOSIT AMT"].fillna(0)


In [ ]:
# Define a new column 'Transaction Type' based on the transaction amounts
# If 'DEPOSIT AMT' is greater than 0, classify the transaction as "Deposit"
# Otherwise, classify it as "Withdrawal"
df["Transaction Type"] = df.apply(lambda row: "Deposit" if row["DEPOSIT AMT"] > 0 else "Withdrawal", axis=1)


In [ ]:
# Extract the year from the 'DATE' column and store it in a new column 'Year'
df["Year"] = df["DATE"].dt.year  

# Extract the month from the 'DATE' column and store it in a new column 'Month'
df["Month"] = df["DATE"].dt.month  


In [ ]:
df # Dataframe (showing the data)

# Load Data

In [ ]:
import sqlite3  # Import SQLite3 to interact with an SQLite database

# Connect to SQLite database (creates the database file 'bank_transactions.db' if it does not exist)
conn = sqlite3.connect("bank_transactions.db")

# Store the transformed DataFrame into an SQL table named 'transactions'
# - if_exists="replace" ensures that if the table already exists, it will be replaced with new data
# - index=False prevents Pandas from writing the DataFrame index as a separate column in the database
df.to_sql("transactions", conn, if_exists="replace", index=False)

# Close the connection to free resources and ensure data is properly written to the database
conn.close()
